In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from manim import *
df = pd.read_csv("wordle.csv")  #reading 5 letter words dataset 
print(df.shape)
df.sample(5)
# Kod kelimelerin sahip olduğu bilgiyi (tüm farklı durumlar için kaç kelime uyduğunun toplamı: entropy) hesaplayıp bu
# kelimeler arasında en yüksek bilgiye sahip olan kelimeyi seçmeyle ve her seçimden sonra elde edilen duruma uymayan 
# kelimelerin çıkarılmasıyla çalışmaktadır.

(12972, 3)


,word,occurrence,day
123,adoze,3.499576e-10,NaN
4183,gales,4.396283e-07,NaN
3982,fores,6.523431e-08,NaN
4909,hauns,1.151461e-09,NaN
1421,brash,4.982697e-07,1417.0


In [46]:
%%manim -qm -v WARNING Updaters
#Bu class anlaşılması daha zor olduğu için diğer kodlara bakabilirsiniz.
#Animasyonların yapıldığı class
class Updaters(Scene):
    results=[]
    result=[0,0,0,0,0]
    best_words=[]
    g_secret_words=[]
    def importdf(self):
        df = pd.read_csv("wordle.csv")  #reading 5 letter words dataset 
        df.sample(5)
        return df
    def find_best_word(self,df,dif_pos): #Sonuca göre en iyi kelimeyi bulma
            ent_list=[]
            for cont_word in df:
                match=np.zeros(243)
                for word in df:
                    used=[]
                    if cont_word!=word:
                        result=[0,0,0,0,0] #kelime result dizisinde sonucu kaydedilip kontrol etmek istediğimiz sonuçla karşılaştırma yapılıyor
                        for i in range(5):
                            if cont_word[i]==word[i]:
                                result[i]=2
                                used.append(i)
                            else:
                                cont=0
                                for j in range(len(cont_word)):
                                    if j not in used and cont_word[j] == word[i]:
                                        cont=1
                                        used.append(j)   
                                if cont==1:
                                    result[i]=1
                        for pos_in in range(len(dif_pos)):
                            if dif_pos[pos_in]==result:
                                match[pos_in]+=1 #match oluşabilecek tüm farklı sonuçların kaç defa gerçekleştiğinin tutulduğu yer
                ent=0
                for i in range(len(match)):
                    if match[i]!=0:
                        size=400
                        ent+=-(match[i]/size)*np.log2(match[i]/size) #entropy hesabı
                ent_list.append(ent) #tüm kelimelerin entropy bilgisi
            index=np.where(ent_list == np.max(ent_list))[0]
            best_word=df[np.random.choice(index)]
            print("best word:",best_word)
            return best_word   
    def find_new(self,main_df,initial_best_word,secret_word): #yeni en iyi kelimeyi bulma
        def pos_control(result,cont_word,word): 
            matches=0
            check=[0,0,0,0,0]
            used=[] #kullanılmış cont_word harflerini tutmak için
            for i in range(5): #ihtimallerin içindeki değerler
                if result[i] == 2 and cont_word[i]==word[i] and cont_word!=word:
                    check[i]=1
                    matches+=1
                    used.append(i)
                elif result[i] == 1 and cont_word!=word and cont_word[i]!=word[i]:#üstteki kontrol elif için yetersiz  
                    cont=0
                    for j in range(len(cont_word)):
                        if j not in used and cont_word[j] == word[i]:
                            cont=1
                            used.append(j)   
                    if cont==1:
                        matches+=1        
                elif result[i] == 0 and word[i] and cont_word!=word:
                    cont=0
                    for j in range(len(cont_word)):
                        if j not in used and cont_word[j] == word[i]:
                            cont=1
                    if cont==0:
                        matches+=1 
            if matches==5:
                return 1
            else:
                return 0
        df=main_df
        best_word=initial_best_word
        found=0
        guess_amount=1
        while found==0:    
            used=[]
            self.result=[0,0,0,0,0]
            for i in range(5):
                if secret_word==best_word:
                    found=1
                    self.result=[2,2,2,2,2]
                elif secret_word[i]==best_word[i]:  #Yazdığımız kelimenin gizli kelimeyle karşılaştırma sonucu
                    self.result[i]=2
                    used.append(i)
                else:
                    cont=0
                    for j in range(len(secret_word)):
                        if j not in used and secret_word[j] == best_word[i]:
                            cont=1
                            used.append(j)   
                    if cont==1:
                        self.result[i]=1
            new_df=[]
            self.results.append(self.result)
            if found!=1:
                for word in df:
                    if(pos_control(self.result,word,best_word) == 1): #elde ettiğimiz sonuç bilgisine 
                        new_df.append(word)                           #(örnek: [0,1,2,1,1]) uyan kelimeler yeni datasetimize kaydediliyor
                df=new_df
                guess_amount+=1
                if(len(df)==1): #yeni dataset 1 elemanlı ise zaten tek bir ihtimal olacağı için direk sonucu bulmuş sayıyoruz.
                    found=1
                    best_word=df[0]
                else:
                    dif_pos=[]      
                    for i in range(3**5): # Oluşabilecek bütün durumların dizisi 0: Harf yok 1: Harf var ama yanlış yerde 2:Harf doğru yerde
                        base_3_num = base_3(i)
                        dif_pos.append(base_3_num) 
                    best_word=self.find_best_word(df,dif_pos) #yeni dataset için en fazla bilgiye sahip kelimeyi bulma
                self.best_words.append(best_word)
        return guess_amount
    def calc(self,df):
        def base_3(n): #gelen değeri 3'lü numara sistemine dönüştürme
            bits = []
            while n > 0: 
                bits.insert(0, n % 3)
                n = n // 3
            while len(bits) < 5: 
                bits.insert(0,0)
            return bits
                
        dif_pos=[]      
        for i in range(3**5): # Oluşabilecek bütün durumların dizisi 0: Harf yok 1: Harf var ama yanlış yerde 2:Harf doğru yerde
            base_3_num = base_3(i)
            dif_pos.append(base_3_num)        
                            
        initial_best_word=self.find_best_word(df,dif_pos)
        return initial_best_word
    def construct(self):
        df=self.importdf()
        df=df["word"]
        df=df.sample(400) 
        df.reset_index(drop=True, inplace=True) 
        df=df.to_numpy()
        secret_words=df[np.random.randint(len(df),size=10)]
        
        self.g_secret_words=secret_words
        avg_guess=[]
        count=0
        word=self.calc(df) #ilk en iyi kelime hesaplanması her seferinde aynı çıkacağı için loop dışında hesaplıyoruz.
        for secret_word in secret_words:
            self.results=[]
            self.result=[0,0,0,0,0]
            self.best_words=[]
            self.g_secret_words=[]
            boxes=VGroup(*[Rectangle(stroke_color= WHITE, fill_color=BLACK, width=0.6, height=0.6) for s in range(0,30)]) 
            #animasyon için kutular
            boxes.arrange_in_grid(rows=6, buff=0.1).to_corner(UL)
            text=Text("Secret word is: "+secret_word).scale(0.5).to_corner(UR)
            self.add(text)
            self.add(boxes)
            self.best_words.append(word)
            guess_amount=self.find_new(df,word,secret_word)
            avg_guess.append(guess_amount)
            count+=1
            self.results.append([2,2,2,2,2])
            w_index=0
            text=Text("Average words to solve is: "+str(np.average(avg_guess))).scale(0.5).to_corner(DR)
            self.add(text)
            for word in self.best_words:
                for i in range(len(word)): #kelimeleri yazma
                    text=Text(word[i]).scale(0.5)
                    text.move_to(boxes[i+w_index*5].get_center())
                    self.play(Create(text),run_time=0.1)
                    animations=[]
                for i in range(len(self.results[0])): #sonuçlara göre kutunun rengini belirleme
                    if(self.results[w_index][i]==2):
                        boxes[i+w_index*5].set_fill(GREEN,opacity=0.8)
                    elif(self.results[w_index][i]==1):
                        boxes[i+w_index*5].set_fill(YELLOW,opacity=0.5)
                self.wait(0.4)
                w_index+=1

            self.clear()
            self.wait(0.1)

                
        return super().construct()

best word: wares
best word: ursid


best word: wilco


best word: repin


best word: labis


best word: goner
best word: tutor


best word: kales
best word: blend


best word: fauna


best word: labis
best word: horal


best word: feres


In [47]:
df=df["word"]
df=df.sample(1000) 
df.reset_index(drop=True, inplace=True)  # Reset the index of the DataFrame
df=df.to_numpy()
def base_3(n): 
    bits = []
    while n > 0: 
        bits.insert(0, n % 3)
        n = n // 3
    while len(bits) < 5: # 15 sayısı için üst satır 1,2,0 değerini döndürüyor lakin ondan önceki değerler yok burada ekleniyor
        bits.insert(0,0)
    return bits
        
dif_pos=[]      
for i in range(3**5): # Oluşabilecek bütün durumların dizisi 0: Harf yok 1: Harf var ama yanlış yerde 2:Harf doğru yerde
    base_3_num = base_3(i)
    dif_pos.append(base_3_num)
    
def pos_control(result,cont_word,word):
    matches=0                           
    check=[0,0,0,0,0]
    used=[] #kullanılmış cont_word harflerini tutmak için
    for i in range(5): #ihtimallerin içindeki değerler
        if result[i] == 2 and cont_word[i]==word[i] and cont_word!=word:
            check[i]=1
            matches+=1
            used.append(i)
        elif result[i] == 1 and cont_word!=word and cont_word[i]!=word[i]:#üstteki kontrol elif için yetersiz  
            cont=0
            for j in range(len(cont_word)):
                if j not in used and cont_word[j] == word[i]:
                    cont=1
                    used.append(j)   
            if cont==1:
                matches+=1        
        elif result[i] == 0 and word[i] and cont_word!=word:
            cont=0
            for j in range(len(cont_word)):
                if j not in used and cont_word[j] == word[i]:
                    cont=1
            if cont==0:
                matches+=1 
    if matches==5:
        return 1
    else:
        return 0
def find_best_word(df,dif_pos): #En iyi kelimeyi bulma
    ent_list=[]                 
    for cont_word in df:
        match=np.zeros(243)
        for word in df:
            used=[]
            if cont_word!=word:
                result=[0,0,0,0,0]
                for i in range(5):
                    if cont_word[i]==word[i]:
                        result[i]=2
                        used.append(i)
                    else:
                        cont=0
                        for j in range(len(cont_word)):
                            if j not in used and cont_word[j] == word[i]:
                                cont=1
                                used.append(j)   
                        if cont==1:
                            result[i]=1
                for pos_in in range(len(dif_pos)):  #elde ettiğimiz sonuç bilgisine 
                    if dif_pos[pos_in]==result:     #(örnek: [0,1,2,1,1]) uyan kelimeler yeni datasetimize kaydediliyor
                        match[pos_in]+=1
        ent=0
        for i in range(len(match)):
            if match[i]!=0:
                size = 1000
                ent+=-(match[i]/size)*np.log2(match[i]/size)
        ent_list.append(ent)
    index=np.where(ent_list == np.max(ent_list))[0]
    best_word=df[np.random.choice(index)]
    print("best word:",best_word)
    return best_word
                    
initial_best_word=find_best_word(df,dif_pos)


best word: tears


In [48]:
main_df=df
secret_words=main_df[np.random.randint(len(main_df),size=10)]

In [49]:

guess_amounts=[]
for secret_word in secret_words:
    df=main_df
    best_word=initial_best_word
    found=0
    guess_amount=1
    while found==0 and secret_word!=best_word:    
        used=[]
        result=[0,0,0,0,0]
        for i in range(5):
            if secret_word==best_word: #Yazdığımız kelimenin gizli kelimeyle karşılaştırma sonucu
                found=1
            elif secret_word[i]==best_word[i]:
                result[i]=2
                used.append(i)
            else:
                cont=0
                for j in range(len(secret_word)):
                    if j not in used and secret_word[j] == best_word[i]:
                        cont=1
                        used.append(j)   
                if cont==1:
                    result[i]=1
        new_df=[]
        if found!=1:
            for word in df:
                if(pos_control(result,word,best_word) == 1):
                    new_df.append(word)
                    #best_word pos_controlden geçerken 0 çıktısını verdiği için 
            df=new_df
            guess_amount+=1
            if(len(df)==1):
                found=1
                best_word=df[0]
            else:
                best_word=find_best_word(df,dif_pos)
        
    guess_amounts.append(guess_amount)
print("Average word to solve is:",str(np.average(guess_amounts)))

best word: lames
best word: axles
best word: deare
best word: finca
best word: lohan
best word: metis
best word: sents
best word: diver
best word: joker
best word: scurs
best word: deems
best word: nelis
best word: pepos
best word: dovie
best word: kyloe
best word: deems
best word: hexes
best word: sexes
best word: bayed
best word: mazed
best word: eaned
best word: faded
Average word to solve is: 3.7
